In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
%matplotlib inline

import glob, os
from pathlib import *
import json

In [2]:
pwd

'/home/lettucestick/github/srs_spotify_bsns/notebooks'

### I: getting paths for data

Here, I build structure into the data to make each file accessible and easier to manipulate using pathlib.  


In [3]:
# using pathlib
BASE_DIR = Path('../').resolve() 
DATA_DIR = BASE_DIR / "data"
NB_DIR = BASE_DIR / "notebooks"

In [4]:
# making list of files
file_list = []

os.chdir(DATA_DIR)
for f in glob.glob("*.json"):
    file_list.append(f)

file_list.sort()

In [5]:
# viewing current list contents
for i,f in enumerate(file_list):
    print(i,f)

0 FamilyPlan.json
1 Follow.json
2 KidsAccount.json
3 Payments.json
4 Playlist1.json
5 SearchQueries.json
6 StreamingHistory0.json
7 StreamingHistory1.json
8 StreamingHistory2.json
9 Userdata.json
10 YourLibrary.json


In [6]:
# opening test json file
with open(file_list[4]) as data_file:
    data = json.load(data_file)

playlist = {}
for name, song in data.items():
    playlist.update(name = song)
    print(len(playlist), name, song)

1 playlists [{'name': 'podcast queue', 'lastModifiedDate': '2019-10-31', 'items': [{'track': None, 'episode': {'episodeName': 'Kamyar Azizzadenesheli', 'showName': 'TalkRL: Reinforcement Learning Interviews'}, 'localTrack': None}, {'track': None, 'episode': {'episodeName': 'S1E5 - Public Wi-Fi - Your Data Is My Data Too ', 'showName': 'Creepy Tech'}, 'localTrack': None}, {'track': None, 'episode': {'episodeName': 'S1E4 - The Dark Side Of Influencer Marketing - No Followers & Still Broke', 'showName': 'Creepy Tech'}, 'localTrack': None}, {'track': None, 'episode': {'episodeName': 'Why the Robot Revolution is Our Fault', 'showName': 'Land of the Giants'}, 'localTrack': None}, {'track': None, 'episode': {'episodeName': 'Why Everyone is Talking About Digital Minimalism ', 'showName': 'Note To Self'}, 'localTrack': None}, {'track': None, 'episode': {'episodeName': 'The Science of Break-Ups', 'showName': 'Stuff You Should Know'}, 'localTrack': None}, {'track': None, 'episode': {'episodeName'

In [7]:
data.keys()

dict_keys(['playlists'])

In [8]:
data['playlists']

[{'name': 'podcast queue',
  'lastModifiedDate': '2019-10-31',
  'items': [{'track': None,
    'episode': {'episodeName': 'Kamyar Azizzadenesheli',
     'showName': 'TalkRL: Reinforcement Learning Interviews'},
    'localTrack': None},
   {'track': None,
    'episode': {'episodeName': 'S1E5 - Public Wi-Fi - Your Data Is My Data Too ',
     'showName': 'Creepy Tech'},
    'localTrack': None},
   {'track': None,
    'episode': {'episodeName': 'S1E4 - The Dark Side Of Influencer Marketing - No Followers & Still Broke',
     'showName': 'Creepy Tech'},
    'localTrack': None},
   {'track': None,
    'episode': {'episodeName': 'Why the Robot Revolution is Our Fault',
     'showName': 'Land of the Giants'},
    'localTrack': None},
   {'track': None,
    'episode': {'episodeName': 'Why Everyone is Talking About Digital Minimalism ',
     'showName': 'Note To Self'},
    'localTrack': None},
   {'track': None,
    'episode': {'episodeName': 'The Science of Break-Ups',
     'showName': 'Stuff 

Now that we've examined the data by hand, we want to put .json information into numpy or pandas data structures/frames for analysis.

In [9]:
def json_to_array(path):
    if path != "":
        json = pd.read_json(path)
        np_array = np.asarray(json)
        print(np_array.shape)  
        return np_array
    
    else:
        print("Error: input must be str(path)")
        

In [10]:
def json_to_df(path):
    if path != "":
        df = pd.read_json(path)
        return df
    
    else:
        print("Error: input must be str(path)")

In [11]:
# to array
hist_1 = json_to_array(file_list[4])
hist_1[1]

(98, 1)


array([{'name': 'transit', 'lastModifiedDate': '2019-10-27', 'items': [{'track': {'trackName': 'Pure Gold', 'artistName': 'half•alive', 'albumName': 'Now, Not Yet'}, 'episode': None, 'localTrack': None}, {'track': {'trackName': 'I See You', 'artistName': 'MISSIO', 'albumName': 'The Darker The Weather // The Better The Man'}, 'episode': None, 'localTrack': None}, {'track': {'trackName': 'Happy Together (feat. Ray Toro)', 'artistName': 'Gerard Way', 'albumName': 'Happy Together (feat. Ray Toro)'}, 'episode': None, 'localTrack': None}, {'track': {'trackName': 'RUNAWAY', 'artistName': 'half•alive', 'albumName': 'Now, Not Yet'}, 'episode': None, 'localTrack': None}, {'track': {'trackName': 'Why Do You Feel So Down', 'artistName': 'Declan McKenna', 'albumName': 'What Do You Think About the Car?'}, 'episode': None, 'localTrack': None}, {'track': {'trackName': 'Gold Guns Girls', 'artistName': 'Metric', 'albumName': 'Fantasies'}, 'episode': None, 'localTrack': None}], 'description': 'what was m

In [12]:
# to df
df_hist = json_to_df(file_list[4])
df_hist.head()

,playlists
0,"{'name': 'podcast queue', 'lastModifiedDate': ..."
1,"{'name': 'transit', 'lastModifiedDate': '2019-..."
2,"{'name': 'school day summers', 'lastModifiedDa..."
3,"{'name': 'bank', 'lastModifiedDate': '2019-10-..."
4,"{'name': 'sunrise', 'lastModifiedDate': '2019-..."


In [13]:
df_hist.columns.values

array(['playlists'], dtype=object)

We see that both np_array and df formats fail to serialize the JSON data appropriately.

So now we'll properly serialize JSON data into a np_array.

In [14]:
# https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

a = np.array([[1, 2, 3], [4, 5, 6]])
print(a.shape)
json_dump = json.dumps({'a': a, 'aa': [2, (2, 3, 4), a], 'bb': [2]}, cls=NumpyEncoder)
print(json_dump)

(2, 3)
{"a": [[1, 2, 3], [4, 5, 6]], "aa": [2, [2, 3, 4], [[1, 2, 3], [4, 5, 6]]], "bb": [2]}


In [15]:
# to restore from JSON

json_load = json.loads(json_dump)
a_restored = np.asarray(json_load["a"])
print(a_restored)
print(a_restored.shape)

[[1 2 3]
 [4 5 6]]
(2, 3)


### II. From playlist to tracks

In [16]:
play_1 = hist_1[1]

In [17]:
play_!

SyntaxError: invalid syntax (<ipython-input-17-3e959965d102>, line 1)

### III. visulization

Visulizing data to inspect initial shapes in 2D.